# Augmention part of Retrieval-Augmented Generation (RAG)

In [ ]:
def get_endpoint_resource_name_and_deployed_index_id():

    try:
        endpoints = aiplatform.MatchingEngineIndexEndpoint.list(
            filter = f'display_name="{INDEX_ENDPOINT_NAME}"'
        )
        endpoint_resource_name = endpoints[0].resource_name
        endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name = endpoint_resource_name
        )
        deployed_index_id = endpoint.deployed_indexes[0].id

        return endpoint_resource_name, deployed_index_id
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

def find_relevant_chunks(query, num_results = 5):
    """
    Takes a user's query, generates an embedding, and finds the most similar chunks in the Vertex AI Vector Search index.
    """

    try:
        response = client.models.embed_content(
            model = GEMINI_EMBEDDING_MODEL,
            contents = query,
            config = types.EmbedContentConfig(task_type = "RETRIEVAL_QUERY"),
        ).embeddings
        embedding_vector = response[0].values
    except Exception as e:
        print(f"Can't generate embeddings for the query: {e}") 

        return

    try:
        endpoint_resource_name, deployed_index_id = get_endpoint_resource_name_and_deployed_index_id()
        index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name = endpoint_resource_name)
        response = index_endpoint.find_neighbors(
            deployed_index_id = deployed_index_id,
            queries = [embedding_vector],
            num_neighbors = num_results
        )
        matched_ids = [neighbor.id for neighbor in response[0]]
        print(f"Found {len(matched_ids)} matching document IDs.")

        return matched_ids
    except Exception as e:
        print(f"Error querying Vertex AI: {e}")

        return

def retrieve_chunks_from_firestore(chunk_ids):
    """
    Retrieves the full text and metadata for a list of chunk IDs from Firestore.
    """

    retrieved_chunks = []
    for chunk_id in chunk_ids:
        doc = get_chunk_in_firestore(chunk_id)
        retrieved_chunks.append(doc)

    return retrieved_chunks